In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import product
from datetime import date
from path import Path
import pandas as pd

from mlbt.run_bt import run_bt

DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)

symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
    "binarize_params": [10, 25, 50, 100],
    "binarize_params": [50],
    "alpha": ["none"],
    "classifier": ["random_forest", "xgboost", "lgbm", "knn", "dummy"],
    "classifier": ["tpot"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
    "func": [run_bt],
#     "skip_feature_imp": [True],
    "check_completed": [False],
#     "optimize_hypers": [False],
#     "load_from_disk": [False],
}

c:\users\doda\anaconda3\envs\fincl\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\doda\anaconda3\envs\fincl\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
2020-02-08 01:45:13,180 Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [3]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas

[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 50,
  'alpha': 'none',
  'classifier': 'tpot',
  'num_threads': 32,
  'n_jobs': 32,
  'func': <function mlbt.run_bt.run_bt(**data)>,
  'check_completed': False}]

In [4]:
len(datas)

1

In [5]:
from mlbt.multiprocess import process_jobs

In [6]:
num_threads = min(32, len(datas))
num_threads
      

1

In [7]:
# process_jobs(datas, num_threads=num_threads)

In [8]:
run_bt(**datas[0])

2020-02-08 01:45:13,776 config: {'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'vol_estimate': 100, 'downsampling': 'cusum', 'symbols': None, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'test_procedure': 'walk_forward', 'classifier': 'tpot', 'bar_type': 'dollar', 'bar_size': None, 'binarize': 'fixed_horizon', 'binarize_params': 50, 'alpha': 'none', 'alpha_params': [], 'feature_calc_only': False, 'feature_imp_only': False, 'skip_feature_imp': False, 'reuse_hypers': True, 'hypers_n_iter': 25, 'load_from_disk': True, 'save_to_disk': True, 'optimize_hypers': True, 'feat_imp_method': 'MDA', 'feat_imp_cv': 5, 'num_threads': 32, 'n_jobs': 32, 'check_completed': False, 'features': [{'name': 'log'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'auto', 'window': 50, 'lag': 25}, {'name': 'stdev', 'window': 50}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'

2020-02-08 01:47:20,274 QCL#C: Feature engineering
2020-02-08 01:47:23,575 QHO#C: Feature engineering
2020-02-08 01:47:26,955 QNG#C: Feature engineering
2020-02-08 01:47:30,492 QPA#C: Feature engineering
2020-02-08 01:47:33,649 QPL#C: Feature engineering
2020-02-08 01:47:36,752 @QG#C: Feature engineering
2020-02-08 01:47:40,488 @QM#C: Feature engineering
2020-02-08 01:47:44,004 QGC#C: Feature engineering
2020-02-08 01:47:47,247 QHG#C: Feature engineering
2020-02-08 01:47:50,511 QSI#C: Feature engineering
2020-02-08 01:47:53,776 @AD#C: Get bins and feature imps
c:\users\doda\anaconda3\envs\fincl\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\doda\pr\fincl\backend\mlbt\run_bt

2020-02-08 01:48:10,236 @W#C: Get bins and feature imps
2020-02-08 01:48:11,194 Dropped label 0.0 0.004821524236591996
2020-02-08 01:48:11,252 bars (106975, 9), events (50449, 2), feats (106975, 37), bins (50156, 3), X_all (43512, 37), X_train (21756, 37)
2020-02-08 01:48:11,257 @MME#C: Get bins and feature imps
2020-02-08 01:48:11,432 Dropped label 0.0 0.001221230624706435
2020-02-08 01:48:11,452 bars (22420, 9), events (10695, 2), feats (22420, 37), bins (10632, 3), X_all (9447, 37), X_train (4723, 37)
2020-02-08 01:48:11,457 @CC#C: Get bins and feature imps
2020-02-08 01:48:12,019 Dropped label 0.0 0.004170113041080783
2020-02-08 01:48:12,057 bars (61619, 9), events (29066, 2), feats (61619, 37), bins (28895, 3), X_all (20521, 37), X_train (10260, 37)
2020-02-08 01:48:12,062 @CT#C: Get bins and feature imps
2020-02-08 01:48:12,578 Dropped label 0.0 0.0016760361744474319
2020-02-08 01:48:12,616 bars (61020, 9), events (28689, 2), feats (61020, 37), bins (28591, 3), X_all (22343, 37),

2020-02-08 01:48:22,223 @KC#C [(14599, 5), (14599, 37), (14599,), (14599, 5), (14599, 37), (14599,)]
2020-02-08 01:48:22,224 @SB#C [(11510, 5), (11510, 37), (11510,), (11511, 5), (11511, 37), (11511,)]
2020-02-08 01:48:22,224 GAS#C [(5195, 5), (5195, 37), (5195,), (5195, 5), (5195, 37), (5195,)]
2020-02-08 01:48:22,224 LF#C [(8739, 5), (8739, 37), (8739,), (8739, 5), (8739, 37), (8739,)]
2020-02-08 01:48:22,224 LG#C [(6370, 5), (6370, 37), (6370,), (6371, 5), (6371, 37), (6371,)]
2020-02-08 01:48:22,225 QCL#C [(11375, 5), (11375, 37), (11375,), (11375, 5), (11375, 37), (11375,)]
2020-02-08 01:48:22,225 QHO#C [(2923, 5), (2923, 37), (2923,), (2924, 5), (2924, 37), (2924,)]
2020-02-08 01:48:22,225 QNG#C [(10837, 5), (10837, 37), (10837,), (10837, 5), (10837, 37), (10837,)]
2020-02-08 01:48:22,226 QPA#C [(5212, 5), (5212, 37), (5212,), (5213, 5), (5213, 37), (5213,)]
2020-02-08 01:48:22,226 QPL#C [(7077, 5), (7077, 37), (7077,), (7077, 5), (7077, 37), (7077,)]
2020-02-08 01:48:22,226 @QG#

Generation 1 - Current best internal CV score: -0.6932017856682207
Generation 2 - Current best internal CV score: -0.693159319700943
Generation 3 - Current best internal CV score: -0.693150594144973
Generation 4 - Current best internal CV score: -0.6929082191032937
Generation 5 - Current best internal CV score: -0.6929082191032937
Generation 6 - Current best internal CV score: -0.692739423194002
Generation 7 - Current best internal CV score: -0.692739423194002
Generation 8 - Current best internal CV score: -0.692739423194002
Generation 9 - Current best internal CV score: -0.692739423194002
Generation 10 - Current best internal CV score: -0.692739423194002
Generation 11 - Current best internal CV score: -0.692739423194002
Generation 12 - Current best internal CV score: -0.6927394231788075
Generation 13 - Current best internal CV score: -0.6927394231788075
Generation 14 - Current best internal CV score: -0.6927394231788075
Generation 15 - Current best internal CV score: -0.69273942317880

TypeError: 'NoneType' object is not iterable

In [ ]:
%debug